In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# import Config
from user_agent import generate_user_agent, generate_navigator
from selenium.common.exceptions import NoSuchElementException
import re
import pandas as pd
from selenium.webdriver.common.by import By
import time


options = webdriver.ChromeOptions()
# user_agent= "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
# user_agent = generate_user_agent(os=('mac'), device_type='desktop')
user_agent = generate_user_agent(device_type='desktop')
options.add_argument('--incognito')
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-setid-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--blink-settings=imagesEnabled=false')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument('user-agent=' + user_agent)
# chrome_path = "/Users/baaam771/Downloads/chromedriver 2"
# driver = webdriver.Chrome(chrome_path)
# driver = webdriver.Chrome(chrome_path, options=options)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
# driver = webdriver.Chrome(executable_path=Config.CONFIG['CHROMEPATH'],options=options)


titles = []
singers = []
numbers = []
LYRIC = []
LYRIC_clean = []
genre_list = []

# for i in range(1,52,50):
for i in range(1,1002,50):
    str_i = str(i)

    driver.get("https://www.melon.com/genre/song_list.htm?gnrCode=GN0100#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex="+str_i)

    time.sleep(0.2)
    title=driver.find_elements(By.CLASS_NAME, 'ellipsis.rank01')

    for a in title:
        titles.append(a.text)

    # print(titles)

    singer =driver.find_elements(By.CLASS_NAME, 'ellipsis.rank02')

    for b in singer:
        singers.append(b.text)

    # print(singers)

    number = driver.find_elements(By.CLASS_NAME, 'btn.button_icons.type03.song_info')
    for i in number:
        numbers.append(re.sub(r'[^0-9]', '', i.get_attribute('href')))


for i in numbers:
    driver.get("https://www.melon.com/song/detail.htm?songId=" + i)
    genre = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]')
    genre_list.append(genre.text)
    try:
        lyric = driver.find_element(By.CLASS_NAME, 'lyric')
        LYRIC.append(lyric.text)
    except NoSuchElementException:
        LYRIC.append("blank")

for i in LYRIC:
    LYRIC_clean.append(i.replace("\n", " "))



df = pd.DataFrame({"melon_music_code":numbers, "제목":titles, "가수":singers, "장르":genre_list, "가사": LYRIC_clean })

df.to_csv("music_data_concat.csv", mode='a', header=False)
df.to_csv("music_data_balad_top_1_new.csv", mode='w')
df.to_excel("music_data_balad_top_1_new.xlsx", encoding='utf-8')

In [ ]:
import pandas as pd
df1 = pd.read_csv('music_data_balad_top_1_new.csv.csv')
df1.isnull().sum()
df2 = df1.iloc[:,1:]
index1 = df2[df2['가사'] == 'blank'].index
df3 = df2.drop(index1)
df4 = df3.drop_duplicates(['melon_music_code'], ignore_index = True)
df4.to_json('music_top1000.json', orient= 'records')